# 간단한 인공 신경망 실습 (pytorch)

In [1]:
import torch  # 핵심 라이브러리(pytorch)
import torch.nn as nn # 뉴런 신경망 기능 포함
import torch.optim as optim # 최적화 부분 함수가 있을 때 최소,최대를 찾을 수 있음 + @
import torchvision # 이미지 처리를 위한 라이브러리
import torchvision.transforms as transforms # 전처리를 위한 라이브러리

## 데이터셋 로드 및 전처리

In [2]:
 # 데이터셋 전처리
transform = transforms.Compose([
    transforms.ToTensor(), # 이미지를 Tensor라고 하는 pytorch에서 사용하는 기본 자료구조로 바꿔줌
    transforms.Normalize((0.5,), (0.5,)) # 이미지 정규화 -> 평균,표준편차 (0.5,0.5)
])

 # MNIST 데이터셋 로드
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

trainset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

일반적으로 pytorch에는 데이터를 다루기 위한 여러가지 기능이 있고, 그 중 데이터를 로드하고 처리하는 부분을 DataLoader라고 하는데 일반적으로 직접 만든다.
Mnist는 일반적으로  torchvision으로 제공되기 때문에 위와같은 "torch.utils.data.DataLoader"함수를 사용 해 주면 편하게 만들 수 있다.

batch_size : 배치 사이즈 만 개의 데이터가 있으면 전체학습을 하기 빡셈. 그래서 size만큼 잘게 쪼갠 단위학습을 함. 속도도 빠르고 학습시간도 빠름 

shuffle : 데이터를 순서대로 쪼개면 순서관계 때문에 의존성이 생길 수 있다. 딥러닝에서의 기본 가정은 데이터간의 독립성을 보장하는 것인데, 이를 위해 섞어서 배치를 만들어라 하는 부분을 지정하는 것




## 간단한 ANN 모델 정의

In [3]:
class SimpleANN(nn.Module): # nn.Module 상속 받은 SimpleANN 클래스 생성
    def __init__(self):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # 입력층에서 은닉층으로
        self.fc2 = nn.Linear(128, 64)       # 은닉층에서 은닉층으로
        self.fc3 = nn.Linear(64, 10)        # 은닉층에서 출력층으로 최종적으로 10 개 예측

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # 입력 이미지를 1차원 벡터로 변환 (fc가 1차원 이니까)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

nn.Moulde을 상속받은 클래스를 만들어 DL을 만들기 위한 기본 기능들을 사전에 가져온다.

Class 내부 구조 :

부모 클래스에서 기능을 작동하는 super 함수를 불러줌. (DL의 기능을 하기 위한 기본함수 불러옴)
***
fc(1,2,3...) : Fully Connected Layer 
각 노드가 이전 레이어의 모든 노드와 연결된 신경망의 한 종류. 이를 통해 네트워크는 입력 데이터의 모든 특징(feature) 사이의 관계를 학습한다.

출력 계산 (1차원들)
y=W⋅x+b 
W: 가중치 행렬 (weight matrix)
x: 입력 벡터 (input vector)
b: 편향 벡터 (bias vector)
y: 출력 벡터 (output vector)

가중치와 편향은 모델이 학습하는 과정에서 조정

***

nn.Linear : ANN을 만드는 함수. layer에서 입력과 출력을 지정해 줘야 한다. 입력 몇개 출력 몇개 << 이런 식으로 why? 이전 layer에서 어떤 것과 연결될 지 모르고 앞으로 어떤 layer와 연결될 지 모르기 때문에 그 정보를 제공해 준다.

nn.Linear(28\*28, 128) 에서 "28*28"은  사용할 Mnist의 `Data크기`라고 생각하면 된다.

***

forward : layer들의 연결관계를 나타내는 함수


 view 살펴보기
 
x.view(-1,28*28) 
view()는 PyTorch에서 텐서의 크기(모양)을 변경하는 함수
`-1`은 "나는 배치 크기를 모른다, 알아서 계산해 줘!" 라고 PyTorch에게 말하는 것이다.
 
 딥러닝 모델은 보통 여러 개의 데이터를 한 번에 처리합니다. 이 여러 개의 데이터를 묶은 것을 **배치(batch)**라고 부릅니다.
예를 들어, 한 번에 64개의 이미지를 모델에 입력한다고 할 때, 이 배치를 표현하면 (64, 28, 28)의 형태를 가집니다. 여기서 64는 배치 크기, 그리고 28, 28은 각각의 이미지의 가로, 세로 크기입니다.

**view() 함수에서 -1의 의미**:

이 배치 데이터를 **Fully Connected Layer**에 전달하기 위해 1차원 벡터 형태로 변환하려고 합니다.
(64, 28, 28) 크기의 텐서를 1차원 벡터로 펼치는 것이 필요합니다. 각 이미지가 28 x 28 = 784이므로, 하나의 이미지당 784개의 값을 가진 벡터로 변환됩니다.

이때, view(-1, 28 * 28)에서 -1의 의미는, 배치 크기를 그대로 유지하라는 뜻입니다. 배치 크기가 몇이든 관계없이, PyTorch가 자동으로 그 값을 계산하도록 지시하는 것입니다.
즉, (64, 28, 28) 크기의 텐서를 view(-1, 28 * 28)로 변환하면, 결과는 (64, 784)가 됩니다.

결과적으로:

-1을 사용하면 PyTorch가 현재 배치 크기를 자동으로 계산해줍니다. 따라서, 우리가 몇 개의 이미지를 가지고 있는지(배치 크기)가 달라지더라도 view(-1, 28 * 28)은 그 크기를 자동으로 맞춰 줍니다.
예를 들어, 배치 크기가 128이라면 (128, 28, 28) 크기의 텐서를 view(-1, 28 * 28)로 변환하면 결과는 (128, 784)가 됩니다.

torch.relu(self.fc1(x)) : 활성화 함수 --> fc1 레이어인 입력 레이어에 x를 전달한 다음 이 출력을 다시 relu를 적용한 값으로 바꾸고 다시 x에 저장함. 

최종적으로 x= self.fc3(x)에서 x = (64,10)이 나올 것이다.
데이터는 64개고 최종 출력 레이어는 10차원에 있기 때문이다.





## 

## 학습 전 모델 예측하기

In [4]:
 # 학습 전 모델 예측
model = SimpleANN()

correct = 0
total = 0
with torch.no_grad(): # 평가단계에서 기울기를 계산하지 않는다.
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


Accuracy of the network on the 10000 test images: 9.76%


torch.no_grad() : 학습을 진행하지 않는다.

왜 torch.max()를 사용할까?
출력 단계에서 10개를 모두 출력했고, 10개의 퍼셉트론에 각 확률이 적혀있다고 생각하면, 그 확률이 가장 큰 값을 찾아줘야 하기 때문에 max함수를 사용

***

모델의 예측 정확도를 계산

```python
total += labels.size(0)
correct += (predicted == labels).sum().item()
```
 
```total += labels.size(0)```
```
labels.size(0):
```
labels는 배치 내의 정답 레이블들을 가지고 있는 텐서다.
size(0)은 텐서의 첫 번째 차원의 크기. 이 첫 번째 차원은 배치 크기를 나타내므로, labels.size(0)은 현재 배치에 포함된 샘플의 수다.
예를 들어, labels가 (64,) 크기를 가진다면 이는 64개의 레이블이 있음을 의미하므로, labels.size(0)은 64입니다.

```total += labels.size(0):```

이 코드는 지금까지 처리한 총 샘플 수를 누적해서 더해가는 역할.
각 배치마다 해당 배치에 포함된 샘플 수를 total에 더함으로써, 전체 데이터셋에서 처리한 총 샘플의 수를 추적하는 것.
이를 통해 전체 데이터셋의 총 레이블 수를 total 변수에 저장.

```correct += (predicted == labels).sum().item()
```


`predicted == labels`:

predicted는 모델이 현재 배치에 대해 예측한 클래스 레이블을 나타내고, labels는 해당 배치에 대한 실제 정답 레이블.
(predicted == labels)는 각 샘플의 예측이 정답과 일치하는지를 비교.
결과는 같은 크기를 가진 **불리언 텐서(Boolean tensor)**로, 맞는 경우 True, 틀린 경우 False 값을 가진다.

예를 들어, predicted = [1, 0, 1]이고 labels = [1, 0, 0]이라면 (predicted == labels)은 [True, True, False]가 된다.

(predicted == labels).sum():

(predicted == labels)의 결과는 불리언 텐서이지만, sum() 함수를 사용하면 True는 1로, False는 0으로 계산됩니다.
이 부분은 올바르게 예측된 샘플의 수를 계산하는 것.
예를 들어, (predicted == labels)의 결과가 [True, True, False]라면, .sum()은 2가 됩니다. 즉, 두 개의 샘플이 올바르게 예측된 것.

.item():

.item()은 파이토치 텐서에서 값을 **Python의 기본 데이터 타입 (예: int, float)**으로 변환해주는 역할.
여기서는 sum()의 결과를 텐서가 아닌 정수 값으로 변환하여 correct에 더한다.
correct += (predicted == labels).sum().item():

이 코드는 지금까지 맞춘 정확한 예측의 개수를 correct 변수에 누적해서 더해가는 역할.
각 배치마다 올바르게 예측된 샘플의 수를 누적하여, 전체 데이터셋에서 정확히 예측된 샘플의 총 개수를 추적.

       

## 모델 학습

In [5]:
# 모델 초기화
model = SimpleANN()  # SimpleANN 클래스의 인스턴스 생성

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()  # 분류 문제에 사용되는 손실 함수로, 모델의 예측 값과 실제 레이블 값 사이의 차이를 계산
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # 경사 하강법(SGD)을 통해 모델의 파라미터를 최적화 (lr: 학습률, momentum: 학습 가속도)

# 모델 학습
epochs = 10  # 전체 데이터셋을 10번 반복 학습 (에포크 수)
for epoch in range(epochs):
    running_loss = 0.0  # 에포크 동안 손실값을 누적하여 로그를 남기기 위함
    
    # 데이터 로더를 통해 배치 단위로 데이터 불러오기
    for i, data in enumerate(trainloader, 0):  # trainloader에서 데이터를 배치 단위로 불러오며 인덱스도 함께 가져옴 (i는 인덱스)
        inputs, labels = data  # 입력 데이터와 실제 레이블을 data에서 분리

        # 기울기 초기화 (이전 배치의 기울기를 제거)
        optimizer.zero_grad()

        # 순전파 + 손실 계산 + 역전파 + 최적화
        outputs = model(inputs)  # 입력 데이터를 모델에 통과시켜 예측 값(outputs) 생성
        loss = criterion(outputs, labels)  # 예측 값과 실제 레이블 간 손실 계산
        loss.backward()  # 역전파를 통해 각 파라미터에 대한 기울기 계산 (손실 값을 기준으로 파라미터 업데이트 방향을 계산)
        optimizer.step()  # 기울기를 바탕으로 가중치 업데이트 (파라미터 최적화)

        # 손실 출력 (중간중간 손실 로그를 남겨 학습 상황을 모니터링)
        running_loss += loss.item()  # 현재 배치의 손실 값을 running_loss에 누적
        if i % 100 == 99:  # 매 100 미니배치마다 손실 값을 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0  # 로그 출력 후 손실 값 초기화

print('Finished Training')

[Epoch 1, Batch 100] loss: 1.344
[Epoch 1, Batch 200] loss: 0.490
[Epoch 1, Batch 300] loss: 0.352
[Epoch 1, Batch 400] loss: 0.342
[Epoch 1, Batch 500] loss: 0.302
[Epoch 1, Batch 600] loss: 0.290
[Epoch 1, Batch 700] loss: 0.268
[Epoch 1, Batch 800] loss: 0.234
[Epoch 1, Batch 900] loss: 0.237
[Epoch 2, Batch 100] loss: 0.198
[Epoch 2, Batch 200] loss: 0.185
[Epoch 2, Batch 300] loss: 0.186
[Epoch 2, Batch 400] loss: 0.183
[Epoch 2, Batch 500] loss: 0.190
[Epoch 2, Batch 600] loss: 0.181
[Epoch 2, Batch 700] loss: 0.155
[Epoch 2, Batch 800] loss: 0.158
[Epoch 2, Batch 900] loss: 0.160
[Epoch 3, Batch 100] loss: 0.136
[Epoch 3, Batch 200] loss: 0.135
[Epoch 3, Batch 300] loss: 0.132
[Epoch 3, Batch 400] loss: 0.127
[Epoch 3, Batch 500] loss: 0.120
[Epoch 3, Batch 600] loss: 0.122
[Epoch 3, Batch 700] loss: 0.124
[Epoch 3, Batch 800] loss: 0.141
[Epoch 3, Batch 900] loss: 0.125
[Epoch 4, Batch 100] loss: 0.111
[Epoch 4, Batch 200] loss: 0.124
[Epoch 4, Batch 300] loss: 0.103
[Epoch 4, 

## 실제 예측

In [7]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


Accuracy of the network on the 10000 test images: 96.83%
